In [42]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
import numpy as np

In [22]:
new_lfc = pd.read_csv('../data/standardized_data/cleaned_ML/lfc_mb_filt.csv')
new_lfc.head()

,Rv_ID,129s1.SvImJ_vs_C57BL.6J,129s1.SvImJ_vs_mbio_H37Rv,A.J_vs_129s1.SvImJ,A.J_vs_C57BL.6J,A.J_vs_mbio_H37Rv,C57BL.6J_vs_129s1.SvImJ,C57BL.6J_vs_A.J,C57BL.6J_vs_C57BL.6J,C57BL.6J_vs_mbio_H37Rv,...,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library,Functional_Category
0,Rv0001,0.00,-0.18,0.00,0.00,-0.18,0.00,0.00,0.0,-0.18,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,information pathways
1,Rv0002,0.00,-0.07,0.00,0.00,-0.07,0.00,0.00,0.0,-0.07,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,information pathways
2,Rv0003,1.54,-1.86,-0.99,0.56,-2.85,-1.54,-0.56,0.0,-3.41,...,-0.31,1.09,-2.13,0.51,0.41,1.30,-0.49,-1.26,-2.25,information pathways
3,Rv0005,-0.56,-0.58,-0.08,-0.64,-0.65,0.56,0.64,0.0,-0.02,...,-0.89,-2.36,-1.89,2.90,2.20,-4.05,0.30,0.69,-3.55,information pathways
4,Rv0006,0.78,2.77,-0.55,0.23,2.22,-0.78,-0.23,0.0,1.99,...,0.23,-1.51,4.80,-2.90,-2.05,-2.93,0.87,3.70,3.67,information pathways


Creating a Y Vector

In [33]:
y_list = []
i = 0
for gene in new_lfc.Functional_Category:
    if gene == "intermediary metabolism and respiration":
        y_list.append(1)
    else:
        y_list.append(0)
    i+=1

y = np.array(y)
y[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

Creating the x vector as a 2D array

In [35]:
raw_x= new_lfc.drop(["Rv_ID", "Functional_Category"], axis = 1)
X = raw_x.to_numpy()
X
print(X.shape)

(2937, 1961)


In [57]:
#Step 1: Create Logistic REgression Classifier.
C_temp = 10
clf = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr', C=C_temp, random_state=42)
#Step 2: Split samples into training and test sets using STratifiedKFOld
n_splits_Temp = 5
n_classes = 2
skf = StratifiedKFold(n_splits=n_splits_Temp, shuffle=True, random_state=42)
confusion = np.zeros((n_classes, n_classes))
#Step 3: ITerate thru each training/test set.
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    #Step 4: Fit the logistic regression to the training data. 
    clf.fit(X_train, y_train)
    #Step 5: Predict the values of the testing data. 
    y_hat = clf.predict(X_test)
    confusion += confusion_matrix(y_test, y_hat)

confusion = normalize(confusion, axis=1, norm='l1')

    

In [56]:
confusion

array([[0.99201597, 0.00798403],
       [0.96034298, 0.03965702]])